In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
data_path = "../dataset/berkley.txt"

In [3]:
with open(data_path) as f:
    data = f.read()
    f.close()

In [4]:
tokens = word_tokenize(data)

In [5]:
punc_list = ["``", "''", ",", ".", "—", "-", ";", ":", "'", "\"", "’"]

In [8]:
l = []
for token in tokens:
    if token in punc_list:
        l.append("</s>")
        l.append("<s>")
    else:
        l.append(token)
l.append("</s>")
l.insert(0, "<s>")

In [9]:
print(l)

['<s>', 'can', 'you', 'tell', 'me', 'about', 'any', 'good', 'cantonese', 'restaurants', 'close', 'by', 'mid', 'priced', 'thai', 'food', 'is', 'what', 'iâ€™m', 'looking', 'for', 'tell', 'me', 'about', 'chez', 'panisse', 'can', 'you', 'give', 'me', 'a', 'listing', 'of', 'the', 'kinds', 'of', 'food', 'that', 'are', 'available', 'iâ€™m', 'looking', 'for', 'a', 'good', 'place', 'to', 'eat', 'breakfast', 'when', 'is', 'caffe', 'venezia', 'open', 'during', 'the', 'day', '</s>']


## Unigram

In [18]:
uni = {}
for i in range(len(l)):
    w = l[i]
    uni[w] = uni.get(w, 0) + 1

In [19]:
uni

{'<s>': 1,
 'can': 2,
 'you': 2,
 'tell': 2,
 'me': 3,
 'about': 2,
 'any': 1,
 'good': 2,
 'cantonese': 1,
 'restaurants': 1,
 'close': 1,
 'by': 1,
 'mid': 1,
 'priced': 1,
 'thai': 1,
 'food': 2,
 'is': 2,
 'what': 1,
 'iâ€™m': 2,
 'looking': 2,
 'for': 2,
 'chez': 1,
 'panisse': 1,
 'give': 1,
 'a': 2,
 'listing': 1,
 'of': 2,
 'the': 2,
 'kinds': 1,
 'that': 1,
 'are': 1,
 'available': 1,
 'place': 1,
 'to': 1,
 'eat': 1,
 'breakfast': 1,
 'when': 1,
 'caffe': 1,
 'venezia': 1,
 'open': 1,
 'during': 1,
 'day': 1,
 '</s>': 1}

## Bigram

In [21]:
bi = {}
for i in range(len(l)-1):
    w = l[i]+ ' '+l[i+1]
    bi[w] = bi.get(w, 0) + 1

In [31]:
## Bigram Probability

bi_prob = {}
for i in bi.keys():
    # print(i.split(" ")[0])
    bi_prob[i] = bi.get(i) / uni[i.split(" ")[0]]

In [32]:
bi_prob

{'<s> can': 1.0,
 'can you': 1.0,
 'you tell': 0.5,
 'tell me': 1.0,
 'me about': 0.6666666666666666,
 'about any': 0.5,
 'any good': 1.0,
 'good cantonese': 0.5,
 'cantonese restaurants': 1.0,
 'restaurants close': 1.0,
 'close by': 1.0,
 'by mid': 1.0,
 'mid priced': 1.0,
 'priced thai': 1.0,
 'thai food': 1.0,
 'food is': 0.5,
 'is what': 0.5,
 'what iâ€™m': 1.0,
 'iâ€™m looking': 1.0,
 'looking for': 1.0,
 'for tell': 0.5,
 'about chez': 0.5,
 'chez panisse': 1.0,
 'panisse can': 1.0,
 'you give': 0.5,
 'give me': 1.0,
 'me a': 0.3333333333333333,
 'a listing': 0.5,
 'listing of': 1.0,
 'of the': 0.5,
 'the kinds': 0.5,
 'kinds of': 1.0,
 'of food': 0.5,
 'food that': 0.5,
 'that are': 1.0,
 'are available': 1.0,
 'available iâ€™m': 1.0,
 'for a': 0.5,
 'a good': 0.5,
 'good place': 0.5,
 'place to': 1.0,
 'to eat': 1.0,
 'eat breakfast': 1.0,
 'breakfast when': 1.0,
 'when is': 1.0,
 'is caffe': 0.5,
 'caffe venezia': 1.0,
 'venezia open': 1.0,
 'open during': 1.0,
 'during the': 

## Predicting Next word

In [38]:
sentence = "a"

In [39]:
for i, j in bi.items():
    if i.split(" ")[0] == sentence:
        print(i, j)

a listing 1
a good 1


In [40]:
text = "This is an example sentence to build a bigram model."

In [43]:
def is_plausible_word(bi_prob, prev_word, word, threshold=0.5):

    bigram = f"{prev_word} {word}"
    if bigram in bi_prob and bi_prob[bigram] >= threshold:
        return True, bi_prob[bigram]
    return False

prev_word = "can"
word1 = "you"  
word2 = "purple"  

print(f"'{word1}' after '{prev_word}':", is_plausible_word(bi_prob, prev_word, word1))
print(f"'{word2}' after '{prev_word}':", is_plausible_word(bi_prob, prev_word, word2))

'you' after 'can': (True, 1.0)
'purple' after 'can': False


In [44]:
text

'This is an example sentence to build a bigram model.'

In [78]:
import numpy as np

def calculate_perplexity(bi_prob, text):
    tokens = text.lower().split()
    total_tokens = len(tokens)
    log_probs = 0
    for i in range(total_tokens - 1):
        prev_word, curr_word = tokens[i], tokens[i + 1]
        bigram = prev_word + " " + curr_word
        if bigram in bi:
            count = bi[bigram]
        else:
            count = 0
        print(count)

In [79]:
perplexity_score = calculate_perplexity(bi_prob, text)
print(f"Perplexity of the bigram model on the text: {perplexity_score}")

0
0
0
0
0
0
0
0
0
Perplexity of the bigram model on the text: None


In [81]:
bi

{'<s> can': 1,
 'can you': 2,
 'you tell': 1,
 'tell me': 2,
 'me about': 2,
 'about any': 1,
 'any good': 1,
 'good cantonese': 1,
 'cantonese restaurants': 1,
 'restaurants close': 1,
 'close by': 1,
 'by mid': 1,
 'mid priced': 1,
 'priced thai': 1,
 'thai food': 1,
 'food is': 1,
 'is what': 1,
 'what iâ€™m': 1,
 'iâ€™m looking': 2,
 'looking for': 2,
 'for tell': 1,
 'about chez': 1,
 'chez panisse': 1,
 'panisse can': 1,
 'you give': 1,
 'give me': 1,
 'me a': 1,
 'a listing': 1,
 'listing of': 1,
 'of the': 1,
 'the kinds': 1,
 'kinds of': 1,
 'of food': 1,
 'food that': 1,
 'that are': 1,
 'are available': 1,
 'available iâ€™m': 1,
 'for a': 1,
 'a good': 1,
 'good place': 1,
 'place to': 1,
 'to eat': 1,
 'eat breakfast': 1,
 'breakfast when': 1,
 'when is': 1,
 'is caffe': 1,
 'caffe venezia': 1,
 'venezia open': 1,
 'open during': 1,
 'during the': 1,
 'the day': 1,
 'day </s>': 1}

In [82]:
import numpy as np

def calculate_perplexity(bi_prob, text, smoothing=1e-6):
    tokens = text.lower().split()
    tokens = ["<s>"] + tokens + ["</s>"]  # Add start and end tokens
    total_tokens = len(tokens)
    log_probs = 0

    for i in range(total_tokens - 1):
        prev_word, curr_word = tokens[i], tokens[i + 1]
        bigram = f"{prev_word} {curr_word}"
        
        # Get the bigram probability, apply smoothing for missing bigrams
        prob = bi_prob.get(bigram, smoothing)
        log_probs += np.log2(prob)

    # Calculate perplexity
    perplexity = 2 ** (-log_probs / (total_tokens - 1))
    return perplexity

# Example usage
text = "This is an example sentence to build a bigram model."
perplexity_score = calculate_perplexity(bi_prob, text)
print(f"Perplexity of the bigram model on the text: {perplexity_score}")

Perplexity of the bigram model on the text: 1000000.0000000024
